In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Div
from bokeh.layouts import row

output_notebook()
pd.options.display.float_format = '{:,.5f}'.format
display(HTML("<style>.container { width:100% !important; }</style>"))
HTML('<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">')

Loading BokehJS ...

Backtesting of a UNI v2 LUNC/BUSD LP strategy
---


In [2]:
from terra_algo_backtest.binance_loader import new_binance_client

# Replace these with your Binance API key and secret
client = new_binance_client(os.getenv("BINANCE_API_KEY"), os.getenv("BINANCE_API_SECRET"))

In [3]:
from binance import Client
from terra_algo_backtest.utils import format_df
from terra_algo_backtest.plotting import new_trade_figure, plot_price_ratio

mkt_pair = 'LUNC/BUSD'
start, end = '2022-10-01 00:00:00', '2023-04-25 23:59:59'
frequency = Client.KLINE_INTERVAL_1HOUR

qty_factor = 0.001
df_trades = client.get_trade_data(mkt_pair, frequency, start, end)
df_trades["quantity"] = qty_factor*df_trades["quantity"]
show(Div(text=format_df(df_trades.head(10), width=800)))

In [5]:
from terra_algo_backtest.market import MarketQuote, MarketPair, new_market, Pool, with_mkt_price
from terra_algo_backtest.simulation import swap_simulation
from terra_algo_backtest.plotting import new_simulation_figure
from terra_algo_backtest.strategy import get_strategy

liquidity_usd = 1000000
# LUNC/BUSD market price
base = MarketQuote("LUNC/USD", df_trades.price.iloc[0])
# LUNC/BUSD market price
quote = MarketQuote("BUSD/USD", 1)
# create a 100000 USD market for LUNC/USTC
mkt = new_market(liquidity_usd, quote, base, 0.003)
# load strategy 
strategy = get_strategy("uni_v2")
# run simulation
simul = swap_simulation(mkt, df_trades, strategy)
# display sim results
show(new_simulation_figure(mkt, simul, plot_height=300))
# show(Div(text=format_df(simul["breakdown"].head(100), width=3200)))

Function 'trade_summary' executed in 0.0074s
Function 'sim_results' executed in 0.0451s
Function 'swap_simulation' executed in 0.3180s
Function 'new_pnl_figure' executed in 0.0235s
Function 'new_portfolio_figure' executed in 0.0360s
Function 'new_price_figure' executed in 0.1497s
Function 'new_fitted_pnl_figure' executed in 0.0453s
Function 'new_sim_price_impact_figure' executed in 0.0217s
Function 'new_roi_distrib_figure' executed in 0.0111s
Function 'new_pnl_arb_figure' executed in 0.0204s


In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

simul["breakdown"].head(100)